## Interpolation Module

## Introduction
To simulate Daedalus measurments using physics based model such as TIEGCM 
values of gridded data have to be interpolateds to the spacecraft's positions.In
Daedalus Maze this is provided by the Interpolation Module which is described below.

The Interpolation Module consists of a family of  codes written in python FORTRAN and C++.

## Basic Input and Output
The Interpolation Module supports Dedalus Input orbits in netcdf format and exports the interpolation
results in netcdf as well.

## Performance and Parallelization Level
The python version of the Inteprolation Modules supports single threaded and multi threaded workloads using the multiprocessing library. At the same, the user can opt to call the pre-compiled low-level FORTRAN subroutines which can handle both the single threaded and the multi threaded interpolations using OpenMP.

The C++ version, uses the OpenMPI and OpenMP interface and can support massive scalability on distributed memory using hybrid programming,splitting the interpolations in threads and nodes.The same basic input and output interface is used as with the python version.

Finally there is a version of the Interpolation Module written in C++ using a CUDA kernel to speed up the interpolation process. On a shared memory environment this is by far the fastest version, outperforming the 
python version by more than three orders of magnitude.






## Interpolation Methods
For obtaining the interpolated data we use two different methods for comparison.These methods are:

1) The linear interpolation method in 3-D space(trilinear) and

2) The cubic spline method in 3-D space(tricubic spline).

Running the code blocks below one can see the differences in these two methods, such as precision and smoothness which is a result of the tricubic spline interpolation.So lets start with the cubic spline.

### Tricubic Spline Interpolation
In this method we use a local (neighbors found using local function in the code below) tricubic interpolation scheme in three dimensions (TriCubicSpline function in the code).The algorithm is based on a specific $ 64 \times 64 $ matrix (matrix A in the code blocks below) that gives the relationship between the derivatives at the corners of the elements and the coefficients of the tricubic interpolant for this element.In contrast with global interpolation where the interpolated function usually depends on the whole data set, our tricubic local interpolation only uses data in a neighbourhood of an element.The tricubic interpolation is always split in three one-dimensional problems.However, in this implementation we use three-dimensional approach.The advantages of this method are: 

1)The reduced computational cost


2)and the availability of accurate derivatives (first and higher orders) of the interpolated data.


The resulting interpolated function and its three first derivatives are continuous if one uses cubic interpolants.

We assume that a function $ f $ is given at the corner of the grid (that is the TIEGCM data grid) and without losing generality we can assume a cubic element of side $ 1 $.We represent the function $ f $ as a piecewise polynomial.Inside each cube, $ f $ takes the values given by the expression:

$$ f(x,y,z) = \sum_{i,j,k=0}^{3} a_{ijk}x^{i}y^{j}z^{k} \tag{1} $$

One achieves $ C^{1} $ continuity if and only if $ f $ and its three first derivatives are continuous on each of the six faces of the cubes.An obvious necessary condition is the continuity of $ f $ and the first derivatives at each of the eight corners p1, p2,...,p8 of the cube.A finite difference method can be used to extract an approximation of the derivatives of $ f $ based on nearby values of $ f $ (FirstOrderDerivative function in the code below).

In this situation, the necessary condition for $ C^{1} $ continuity results in four constraints at the eight corners of each element, giving $ 32 $ constraints.We will derive the equations for the coefficients $ a_{ijk} $ in such a way that they satisfy the necessary condition ($ C^{1} $ continuity at the eight corners).

The tricubic interpolation form given by Equation ($ 1 $) uses $ 64 $ coefficients and enforcing $ C^{1} $
continuity at the eight corners only provides $ 32 $ constraints.Now we need another set of $ 32 $ constraints that must satisfy the $ C^{1} $ continuity and also be invariant under rotations of the axis x, y and z (isotropic).

The choice that satisfy the above requirements and guarantees existence and uniqueness of the solution is:

$$\left[ \frac{\partial^{2}{f}}{{\partial{x} \partial{y}}}, \frac{\partial^{2}{f}}{{\partial{x} \partial{z}}}, \frac{\partial^{2}{f}}{{\partial{y} \partial{z}}}, \frac{\partial^{3}{f}}{{\partial{x} \partial{y}} \partial{z}}  \right] \tag{2}$$

Therefore, the coefficients $ a_{ijk} $ will be determined for each element in such a way that the values of each function in the set:

$$ \left[ f,\frac{\partial{f}}{{\partial{x}}},\frac{\partial{f}}{{\partial{y}}},\frac{\partial{f}}{{\partial{z}}},\frac{\partial^{2}{f}}{{\partial{x} \partial{y}}}, \frac{\partial^{2}{f}}{{\partial{x} \partial{z}}}, \frac{\partial^{2}{f}}{{\partial{y} \partial{z}}}, \frac{\partial^{3}{f}}{{\partial{x} \partial{y}} \partial{z}} \right] \tag{3}$$
take prescribed values at the eight corners of the element.

We stack the 64 coefficients $ a_{ijk} $ of the interpolant in Equation ($ 3 $) in a vector $ \vec{\alpha} $ (array a in the code below sized $ 64 $) by defining 

$$ \alpha_{i+1+4j+16k} = a_{ijk} \> \> for \> \> all \> \> i,j,k \in \{0,1,2,3\} \tag{4}$$


Similarly, we give a unique index p1, p2,...,p8 to each corner of the cube.We stack the constraints on $ f $ and its derivatives in a vector $\vec{b}$ (in the code below is shown as x array) by defining:

$$ b_{i}=\begin{cases} 
f(p_{i}), & \text{if}\ 1 \leq i \leq 8 \\ 
\frac{\partial{f}}{\partial{x}}(p_{i-8}), & \text{if }\ 9 \leq i \leq 16 \\
\frac{\partial{f}}{\partial{y}}(p_{i-16}), & \text{if }\ 17 \leq i \leq 24 \\
\frac{\partial{f}}{\partial{z}}(p_{i-24}), & \text{if }\ 25 \leq i \leq 32 \\
\frac{\partial^{2}{f}}{\partial{x} \partial{y}}(p_{i-32}), & \text{if }\ 33 \leq i \leq 40 \\
\frac{\partial^{2}{f}}{\partial{x} \partial{z}}(p_{i-40}), & \text{if }\ 41 \leq i \leq 48 \\
\frac{\partial^{2}{f}}{\partial{y} \partial{z}}(p_{i-48}), & \text{if }\ 49 \leq i \leq 56 \\
\frac{\partial^{3}{f}}{\partial{x} \partial{y} \partial{z}}(p_{i-56}), & \text{if }\ 57 \leq i \leq 64 \\
\end{cases} \tag{5}$$

Based on Equation ($ 1 $), the derivatives of $ f $ can be computed and evaluated for the eight points $ p_{i} $.This gives a linear system in the $ 64 $ unknown coefficients $ \alpha_{i}$ of the form:

$$ A\vec{\alpha} = \vec{b} \tag{6} $$, where the A matrix is too large to be shown but its elements are integers and are computed exactly (i.e without numerical error) and can be found in the code below.

The determinant of the matrix A is:
$\det(A)=1$ and so the matrix A is invertible and the coefficients $ \alpha_{i} $ can be computed using the linear relationship:

$$ \vec{\alpha} = A^{-1}\vec{b} \tag{7} $$


### Trilinear Interpolation
As in the method above we can consider a cubic element of side 1 for simplicity, without loss in generality.First, for each point we calculate its neighbors (local function in the code below).Then, lets define the differences $ dx, dy, dz $:

$$ dx = \frac{x-x_{0}}{x_{1}-x_{0}} \\
dy = \frac{y-y_{0}}{y_{1}-y_{0}} \\
dz = \frac{z-z_{0}}{z_{1}-z_{0}} $$, 
where in the code below $ x $ stands for altitude, $ y $ stands for latitude and $ z $ for longitude and $ x-x_{0} $ corresponds to dalt[i]-alts[r_local] and $ x_{1}-x_{0}$ to deltarho.The other variables are in the same way.Then, for each dimension we must define the corresponding weights depending on which neighbor the point is closer, as seen below:

* $$ w_{1} = (1-dx)(1-dy)(1-dz)$$ 


* $$ w_{2} = dx \> (1-dy)(1-dz) $$ 


* $$ w_{3} = (1-dx) \> dy \> (1-dz) $$


* $$ w_{4} = dx \> dy \> (1-dz) $$


* $$ w_{5} = (1-dx)(1-dy) \> dz $$


* $$ w_{6} = dx \> (1-dy) \> dz $$


* $$ w_{7} = (1-dx) \> dy \> dz $$


* $$ w_{8} = dx \> dy \> dz $$

If we define each corner of the cube as $ C_{i,j,k} $, where $i \> corresponds \> to \>  x, \>  j \>  to  \> y \>  and  \> k  \> to  \> z$, then we get the final equation:

$$ C(x,y,z) = C_{000} \> w_{1} + C_{100} \> w_{2} + C_{010} \> w_{3} + C_{110} \> w_{4} + C_{001} \> w_{5} + C_{101} \> w_{6} + C_{011} \> w_{7} + C_{111} \> w_{8} $$

The corners of the cube above ($ C_{i,j,k} $) are the neighbors of the interpolated point in the TIEGCM data grid.


### References
[1] Trilinear Interpolation, https://www.wikiwand.com/en/Trilinear_interpolation


[2] Tricubic Interpolation, Tricubic interpolation in three dimensions,F. Lekien and J. Marsden, INTERNATIONAL JOURNAL FOR NUMERICAL METHODS IN ENGINEERING Int. J. Numer. Meth. Engng 2005; 63:455–471

## Skeleton of the Interpolation Module


## The Model Class: 
This class handles basic IO of the Model Data extracting grid quintities and variables as selected by
the user to be used in the interpolation.


In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
%matplotlib inline  
import pandas as pd
import numpy as np
import pandas as pd
from pandas import DataFrame
from datetime import datetime
import time
import os
import sys
import multiprocessing
from functools import partial
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')

startTime = 0

In [ ]:

class Model:
    def __init__(self,name, maxAltitude, minAltitude):
        self.name=name
        self.maxAltitude=maxAltitude
        self.minAltitude=minAltitude
        self.dt=0.0



    def readGrid(self,name):
        TIEGCM=Dataset(name+".nc")
        glat=TIEGCM.variables['lat'][:]
        glon=TIEGCM.variables['lon'][:]
        glev=TIEGCM.variables['lev'][:]
        gtime=TIEGCM.variables['time'][:] 
        zg=TIEGCM.variables['ZGMID'][:]
        TIEGCM.close()
        self.dt= gtime[2]- gtime[1]
       
        return np.asarray(gtime),np.asarray(glat),np.asarray(glon),np.asarray(glev),np.asarray(zg)

    def readVar(self,model,name):
        TIEGCM=Dataset(model+".nc")
        var=TIEGCM.variables[name][:]
        TIEGCM.close()
        return var


## The Orbit Class: 
This class  This class handles basic IO of Daedalus' orbit allocating arrays for the spatial components. The longitudinal component is matched to TIEGCM's  and points in a specific altitudinal range are extracted to be passed to the interpolation routine. There is also an option for creating a Rocket orbit for getting 
vertical profiles of the said model.

In [ ]:

class Orbit:
    def __init__(self,name):
        self.name=name
        self.dt=1/16



    def createorbit(self,filename,minAlt,maxAlt,outfile,Save=False):
        orbit=Dataset(filename+".nc","r")
        daed_lat_temp = orbit.variables['lat'][:]
        daed_lon_temp = orbit.variables['lon'][:]
        daed_alt_temp = orbit.variables['altitude'][:]
        daed_time_temp = orbit.variables['time'][:] #unix time
        
        global startTime
        startTime = daed_time_temp[0]
        

#         self.dt=daed_time_temp[2]-daed_time_temp[1]
        self.Save=Save
        
        if self.Save==True:
            self.OpenFile(outfile,daed_time_temp,daed_lat_temp,daed_lon_temp,daed_alt_temp)
        
        stop=False
        counter=0
        for i in range(0,len(daed_alt_temp)):
            if (daed_alt_temp[i] < maxAlt and daed_alt_temp[i] > minAlt):
                counter=counter+1

        
        
        daed_lat=np.zeros((counter))
        daed_lon=np.zeros((counter))
        daed_alt=np.zeros((counter))
        daed_time=np.zeros((counter),dtype=datetime)
        index=[None]*counter
        int_final=[None]*len(daed_alt_temp)


   
        counter2=0
        
        for i in range(0,len(daed_alt_temp)):
    
            if (daed_alt_temp[i] < maxAlt and daed_alt_temp[i] > minAlt):
         
                daed_time[counter2]=daed_time_temp[i]
                daed_lat[counter2]=daed_lat_temp[i]
                if daed_lon_temp[i]>180:
                    daed_lon[counter2]=daed_lon_temp[i]-360 #match TIEGCM grid
                else:
                    daed_lon[counter2]=daed_lon_temp[i]
                daed_alt[counter2]=daed_alt_temp[i]
                
                index[counter2]=i          #keep indices for merging data
                counter2=counter2+1
        

        return(daed_time,daed_lat,daed_lon,daed_alt,index,int_final)

    def RockeOrbit(self,lat,lon,alt,nop,dz):
        Rlat=np.zeros(nop)
        Rlon=np.zeros(nop)
        Ralt=np.zeros(nop)

        Rlat[:]=lat
        Rlon[:]=lon
        Ralt[0]=alt
        for i in range(1,nop):
            Ralt[i]=Ralt[i-1]+dz

        return Rlat,Rlon,Ralt

    def mergeData(self,index,int_final,m):
        for i in range(0,len(m)):
            int_final[index[i]]=m[i]

        return int_final    



    def OpenFile(self,outfile,time,lat,lon,alt):
        print("Output File Path:",outfile)
        ncout = Dataset(outfile, "w", format="NETCDF4")    
        ncout.createDimension("time", len(time))
        xaxis=np.arange(0,len(time)) #time axis based on Daedalus Sampling rate
        data = ncout.createVariable("time","f4","time")
        data[:]=xaxis
        ncout.createDimension("lat",len(lat))
        data = ncout.createVariable("lat","f4","lat")
        data[:]=lat
        ncout.createDimension("lon",len(lon))
        data = ncout.createVariable("lon","f4","lon")
        data[:]=lon
        ncout.createDimension("altitude",len(alt))
        data = ncout.createVariable("altitude","f4","time")
        data[:]=alt
        ncout.close()   
        return True



Functions used by the Interpolation Module
The geod2geo_lat converts geodetic latitude to geographic latitude

In [ ]:
def geod_lat2geo_lat(phi):

    # calculate geocentric latitude from geodetic latitude
    # according to WGS 84
    a = 6378137  # meter semi major axis of earth
    f = 1 / 298.257  # flattening
    b = a - f * a  # semi minor axis
    e = ((a ** 2 - b ** 2) ** (1 / 2)) / a
    phi_rad = np.deg2rad(phi)
    geo_lat = np.arctan((1 - e ** 2) * np.tan(phi_rad))
    geo_lat = np.rad2deg(geo_lat)
    return geo_lat  # in degrees

The function local is used to find the local neighbors of a specific component in a non eqwually spaced array 
such as the pressure levels in the TIEGCM

In [ ]:

def local(dim1,y,x):
    local_pos=0
    for i in range(0,len(x)-1):
        if y >= x[i] and y < x[i+1]:
            if y<=0:
                local_pos=i+1
                return (local_pos)
            else:
                local_pos=i
                return (local_pos)
    return (local_pos)

The following functions are used on order to calculate first,second and thrird order partial derivatives. Moreover table "A" carries Tricubic Spline Coefficients.

In [ ]:
A = [
[ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-3, 3, 0, 0, 0, 0, 0, 0,-2,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 2,-2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 9,-9,-9, 9, 0, 0, 0, 0, 6, 3,-6,-3, 0, 0, 0, 0, 6,-6, 3,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-6, 6, 6,-6, 0, 0, 0, 0,-3,-3, 3, 3, 0, 0, 0, 0,-4, 4,-2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-2,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-6, 6, 6,-6, 0, 0, 0, 0,-4,-2, 4, 2, 0, 0, 0, 0,-3, 3,-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-1,-2,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 4,-4,-4, 4, 0, 0, 0, 0, 2, 2,-2,-2, 0, 0, 0, 0, 2,-2, 2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 3, 0, 0, 0, 0, 0, 0,-2,-1, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,-2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9,-9,-9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 3,-6,-3, 0, 0, 0, 0, 6,-6, 3,-3, 0, 0, 0, 0, 4, 2, 2, 1, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-6, 6, 6,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3,-3, 3, 3, 0, 0, 0, 0,-4, 4,-2, 2, 0, 0, 0, 0,-2,-2,-1,-1, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-6, 6, 6,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4,-2, 4, 2, 0, 0, 0, 0,-3, 3,-3, 3, 0, 0, 0, 0,-2,-1,-2,-1, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,-4,-4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,-2,-2, 0, 0, 0, 0, 2,-2, 2,-2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
[-3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 9,-9, 0, 0,-9, 9, 0, 0, 6, 3, 0, 0,-6,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6,-6, 0, 0, 3,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-6, 6, 0, 0, 6,-6, 0, 0,-3,-3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4, 4, 0, 0,-2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-2, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0, 0, 0,-1, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9,-9, 0, 0,-9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 3, 0, 0,-6,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6,-6, 0, 0, 3,-3, 0, 0, 4, 2, 0, 0, 2, 1, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-6, 6, 0, 0, 6,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3,-3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4, 4, 0, 0,-2, 2, 0, 0,-2,-2, 0, 0,-1,-1, 0, 0],
[ 9, 0,-9, 0,-9, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 3, 0,-6, 0,-3, 0, 6, 0,-6, 0, 3, 0,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 9, 0,-9, 0,-9, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 3, 0,-6, 0,-3, 0, 6, 0,-6, 0, 3, 0,-3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 2, 0, 1, 0],
[-27,27,27,-27,27,-27,-27,27,-18,-9,18, 9,18, 9,-18,-9,-18,18,-9, 9,18,-18, 9,-9,-18,18,18,-18,-9, 9, 9,-9,-12,-6,-6,-3,12, 6, 6, 3,-12,-6,12, 6,-6,-3, 6, 3,-12,12,-6, 6,-6, 6,-3, 3,-8,-4,-4,-2,-4,-2,-2,-1],
[18,-18,-18,18,-18,18,18,-18, 9, 9,-9,-9,-9,-9, 9, 9,12,-12, 6,-6,-12,12,-6, 6,12,-12,-12,12, 6,-6,-6, 6, 6, 6, 3, 3,-6,-6,-3,-3, 6, 6,-6,-6, 3, 3,-3,-3, 8,-8, 4,-4, 4,-4, 2,-2, 4, 4, 2, 2, 2, 2, 1, 1],
[-6, 0, 6, 0, 6, 0,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0,-3, 0, 3, 0, 3, 0,-4, 0, 4, 0,-2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-2, 0,-1, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0,-6, 0, 6, 0, 6, 0,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 0,-3, 0, 3, 0, 3, 0,-4, 0, 4, 0,-2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-2, 0,-1, 0,-1, 0],
[18,-18,-18,18,-18,18,18,-18,12, 6,-12,-6,-12,-6,12, 6, 9,-9, 9,-9,-9, 9,-9, 9,12,-12,-12,12, 6,-6,-6, 6, 6, 3, 6, 3,-6,-3,-6,-3, 8, 4,-8,-4, 4, 2,-4,-2, 6,-6, 6,-6, 3,-3, 3,-3, 4, 2, 4, 2, 2, 1, 2, 1],
[-12,12,12,-12,12,-12,-12,12,-6,-6, 6, 6, 6, 6,-6,-6,-6, 6,-6, 6, 6,-6, 6,-6,-8, 8, 8,-8,-4, 4, 4,-4,-3,-3,-3,-3, 3, 3, 3, 3,-4,-4, 4, 4,-2,-2, 2, 2,-4, 4,-4, 4,-2, 2,-2, 2,-2,-2,-2,-2,-1,-1,-1,-1],
[ 2, 0, 0, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-6, 6, 0, 0, 6,-6, 0, 0,-4,-2, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 3, 0, 0,-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2,-1, 0, 0,-2,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 4,-4, 0, 0,-4, 4, 0, 0, 2, 2, 0, 0,-2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,-2, 0, 0, 2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-6, 6, 0, 0, 6,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4,-2, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-3, 3, 0, 0,-3, 3, 0, 0,-2,-1, 0, 0,-2,-1, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,-4, 0, 0,-4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0,-2,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,-2, 0, 0, 2,-2, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
[-6, 0, 6, 0, 6, 0,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4, 0,-2, 0, 4, 0, 2, 0,-3, 0, 3, 0,-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0,-2, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0,-6, 0, 6, 0, 6, 0,-6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-4, 0,-2, 0, 4, 0, 2, 0,-3, 0, 3, 0,-3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,-2, 0,-1, 0,-2, 0,-1, 0],
[18,-18,-18,18,-18,18,18,-18,12, 6,-12,-6,-12,-6,12, 6,12,-12, 6,-6,-12,12,-6, 6, 9,-9,-9, 9, 9,-9,-9, 9, 8, 4, 4, 2,-8,-4,-4,-2, 6, 3,-6,-3, 6, 3,-6,-3, 6,-6, 3,-3, 6,-6, 3,-3, 4, 2, 2, 1, 4, 2, 2, 1],
[-12,12,12,-12,12,-12,-12,12,-6,-6, 6, 6, 6, 6,-6,-6,-8, 8,-4, 4, 8,-8, 4,-4,-6, 6, 6,-6,-6, 6, 6,-6,-4,-4,-2,-2, 4, 4, 2, 2,-3,-3, 3, 3,-3,-3, 3, 3,-4, 4,-2, 2,-4, 4,-2, 2,-2,-2,-1,-1,-2,-2,-1,-1],
[ 4, 0,-4, 0,-4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,-2, 0,-2, 0, 2, 0,-2, 0, 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,-4, 0,-4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,-2, 0,-2, 0, 2, 0,-2, 0, 2, 0,-2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
[-12,12,12,-12,12,-12,-12,12,-8,-4, 8, 4, 8, 4,-8,-4,-6, 6,-6, 6, 6,-6, 6,-6,-6, 6, 6,-6,-6, 6, 6,-6,-4,-2,-4,-2, 4, 2, 4, 2,-4,-2, 4, 2,-4,-2, 4, 2,-3, 3,-3, 3,-3, 3,-3, 3,-2,-1,-2,-1,-2,-1,-2,-1],
[ 8,-8,-8, 8,-8, 8, 8,-8, 4, 4,-4,-4,-4,-4, 4, 4, 4,-4, 4,-4,-4, 4,-4, 4, 4,-4,-4, 4, 4,-4,-4, 4, 2, 2, 2, 2,-2,-2,-2,-2, 2, 2,-2,-2, 2, 2,-2,-2, 2,-2, 2,-2, 2,-2, 2,-2, 1, 1, 1, 1, 1, 1, 1, 1]
]
A=np.asarray(A)
# i  j   k   z
def handleBC(time, r, theta, phi):
    i = time
    j = r
    k = theta
    z = phi
    # return i,j,k,z

    isLonInBC = (phi > 143 or phi < 0)
    isLatInBC = (theta > 71 or theta < 0)

    # if (not (isLonInBC and isLatInBC)):
    if (True):

        # print("Correcting")
        # elevation BCs  (lat->theta) [0-71]
        dletaphi=2.5
        if theta == 72:
            k = 71
            sign = phi > 71
            if (sign):
                z -= int(180 // deltaphi)
            else:
                z += int(180 // deltaphi)

        elif theta == 73:
            k = 70
            sign = phi > 71
            if (sign):
                z -= int(180 // deltaphi)
            else:
                z += int(180 // deltaphi)


        elif theta == -1:
            k = 0
            sign = phi > 71
            if (sign):
                z -= int(180 // deltaphi)
            else:
                z += int(180 // deltaphi)


        elif theta == -2:
            k = 1
            sign = phi > 71
            if (sign):
                z -= int(180 // deltaphi)
            else:
                z += int(180 // deltaphi)

        if phi == 144:
            # print("1")
            z = 0
        elif phi == 145:
            # print("2")
            z = 1
        elif phi == -1:
            # print("3")
            z = 143
        elif phi == -2:
            # print("corr")
            z = 142

    return i, j, k, z
def FirstOrderDerivative(f,glat,glon,alts,time,x,y,z,mode):
  
    if mode == "X":
        retval = (f[handleBC(time, x + 1, y, z)] - f[handleBC(time, x - 1, y, z)]) / (2)

        return retval

    if mode == "Y":
        retval = (f[handleBC(time, x, y + 1, z)] - f[handleBC(time, x, y - 1, z)]) / (2)
        return retval

    if mode == "Z":
        retval = (f[handleBC(time, x, y, z + 1)] - f[handleBC(time, x, y, z - 1)]) / (2)

        return retval

    return False   


def SecondOrderMixedDerivative(f,glat,glon,alts,time,x,y,z,mode):
 
    if mode == "XY":
        retval = (f[handleBC(time, x + 1, y + 1, z)] - f[handleBC(time, x + 1, y - 1, z)] - f[
            handleBC(time, x - 1, y + 1, z)] + f[handleBC(time, x - 1, y - 1, z)]) / (4)
        return retval

    if mode == "XZ":
        retval = (f[handleBC(time, x + 1, y, z + 1)] - f[handleBC(time, x + 1, y, z - 1)] - f[
            handleBC(time, x - 1, y, z + 1)] + f[handleBC(
            time, x - 1, y, z - 1)]) / (4)
        return retval

    if mode == "YZ":
        retval = (f[handleBC(time, x, y + 1, z + 1)] - f[handleBC(time, x, y + 1, z - 1)] - f[
            handleBC(time, x, y - 1, z + 1)] + f[handleBC(
            time, x, y - 1, z - 1)]) / (4)
        return retval

    return False



def ThirdOrderMixedDerivative(f,glat,glon,alts,time,x,y,z,mode):

    retval = (f[handleBC(time, x + 1, y + 1, z + 1)] - f[handleBC(time, x + 1, y + 1, z - 1)] - f[
    handleBC(time, x + 1, y - 1, z + 1)] + f[handleBC(
    time, x + 1, y - 1, z - 1)] - f[handleBC(time, x - 1, y + 1, z + 1)] + \
          f[handleBC(time, x - 1, y + 1, z - 1)] + f[handleBC(time, x - 1, y - 1, z + 1)] - f[
              handleBC(time, x - 1, y - 1, z - 1)]) / (
             8)

    return retval

The single threaded inteprolation core routine which iterates over Daedalus orbit and performs the interpolation.
At every spatial position weights are assigned to the  8 local grid neighbours and then the interpolated value is calculated using trilinear interpolation.

In [ ]:
from tqdm import tqdm
def Interpolate_Serial(counter,glat,glon,glev,dtime,dlat,dlon,dalt,zg,ne,Plot=False,thisLabel="TN"):
#     counter=11
    deltaphi= np.abs(glon[2]-glon[1])
    deltatheta=np.abs(glat[2]-glat[1])
    Re=6378137.0/1e3
    # arc_theta=2*np.pi*Re*(deltatheta/360)
    # arc_phi=2*np.pi*Re*(deltaphi/360)

    m=np.zeros((len(dalt)))
    tn=np.zeros((len(dalt)))
    pn=np.zeros((len(dalt)))

    for i in range(0,len(dalt)):


        
        phi_local=local(len(glon),dlon[i],glon)
        theta_local=local(len(glat),dlat[i],glat)


        if dlon[i] >= 177.5 :
            phi_local=len(glon)-2
            lonbnd=1

        if dlon[i] <= -177.5:
            phi_local=1

        if dlat[i] >= 87.5:
            theta_local=len(glat)-2
            latbnd=1

        if dlat[i] <= -87.5:
            theta_local=0


        alts=zg[counter,:,theta_local,phi_local]/1e5
        r_local=local(len(alts),dalt[i],alts)
        deltarho=alts[r_local+1]-alts[r_local]
        dx=(((dalt[i]-alts[r_local])/deltarho))
        dy=(((dlat[i]-glat[theta_local])/deltatheta))
        dz=np.abs(((dlon[i]-glon[phi_local])/deltaphi))

        # Calculate Neighbors
        pn[i]=ne[counter,r_local,theta_local,phi_local]

        
        w1=np.abs((1-dx)*(1-dy)*(1-dz))
        w2=np.abs((dx)*(1-dy)*(1-dz))
        w3=np.abs((1-dx)*(dy)*(1-dz))
        w4=np.abs((dx)*(dy)*(1-dz))
        w5=np.abs((1-dx)*(1-dy)*(dz))
        w6=np.abs((dx)*(1-dy)*(dz))
        w7=np.abs((1-dx)*(dy)*(dz))
        w8=np.abs((dx)*(dy)*(dz))

        
        m[i]=0.0
        m[i]=       ne[counter,r_local,theta_local,phi_local]*w1
        m[i]=m[i]+  ne[counter,r_local+1,theta_local,phi_local]*w2
        m[i]=m[i]+  ne[counter,r_local,theta_local+1,phi_local]*w3
        m[i]=m[i]+  ne[counter,r_local+1,theta_local+1,phi_local]*w4

        m[i]=m[i]+  ne[counter,r_local,theta_local,phi_local+1]*w5
        m[i]=m[i]+  ne[counter,r_local+1,theta_local,phi_local+1]*w6
        m[i]=m[i]+  ne[counter,r_local,theta_local+1,phi_local+1]*w7
        m[i]=m[i]+  ne[counter,r_local+1,theta_local+1,phi_local+1]*w8



    return (m)


def TriCubicSpline(counter,glat,glon,glev,dtime,dlat,dlon,dalt,zg,ne,Plot=False,thisLabel="TN",srt=1):
#     counter=19
    deltaphi= np.abs(glon[2]-glon[1])
    deltatheta=np.abs(glat[2]-glat[1])
    Re=6378137.0/1e3
    # arc_theta=2*np.pi*Re*(deltatheta/360)
    # arc_phi=2*np.pi*Re*(deltaphi/360)

    m=np.zeros((len(dalt)))
    m2 = np.zeros((len(dalt)))
    tn=np.zeros((len(dalt)))
    pn=np.zeros((len(dalt)))
    fval=np.zeros(8)
    fdx=np.zeros(8)
    fdy=np.zeros(8)
    fdz=np.zeros(8)
    fdxdy=np.zeros(8)
    fdxdz=np.zeros(8)
    fdydz=np.zeros(8)
    fdxdydz=np.zeros(8)
    x=np.zeros(64)
    global geitonas
    geitonas=np.zeros((len(dalt)))
#     dlat[:]=56
    for i in range(0,len(dalt)):
    
        phi_local=local(len(glon),dlon[i],glon)
        theta_local=local(len(glat),dlat[i],glat)

        if dlon[i] >= 177.5 :
            phi_local=len(glon)-2
            lonbnd=1

        if dlon[i] <= -177.5:
            phi_local=1

        if dlat[i] >= 87.5:
            theta_local=len(glat)-2
            latbnd=1

        if dlat[i] <= -87.5:
            theta_local=0



        alts=zg[counter,:,theta_local,phi_local]/1e5
        r_local=local(len(alts),dalt[i],alts)
        deltarho=alts[r_local+1]-alts[r_local]
        dx=(((dalt[i]-alts[r_local])/deltarho))
        dy=np.abs((((dlat[i]-glat[theta_local])/deltatheta)))
        dz=np.abs(((dlon[i]-glon[phi_local])/deltaphi))
        
         
                


        

        # Get 8 Nearest Neighbors Values
        fval[0]=ne[counter,r_local,theta_local,phi_local]
        fval[1]=ne[counter,r_local+1,theta_local,phi_local]
        fval[2]=ne[counter,r_local,theta_local+1,phi_local]
        fval[3]=ne[counter,r_local+1,theta_local+1,phi_local]
        fval[4]=ne[counter,r_local,theta_local,phi_local+1]
        fval[5]=ne[counter,r_local+1,theta_local,phi_local+1]
        fval[6]=ne[counter,r_local,theta_local+1,phi_local+1]
        fval[7]=ne[counter,r_local+1,theta_local+1,phi_local+1]
        
        geitonas[i]=ne[counter,r_local,theta_local,phi_local]



        # Get 1st Order Derivatives Nearest Neighbors Values
        fdx[0]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"X")  
        fdx[1]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"X")  
        fdx[2]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"X")  
        fdx[3]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"X")  
        fdx[4]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"X")  
        fdx[5]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"X")  
        fdx[6]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"X")  
        fdx[7]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"X")  

        fdy[0]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"Y")  
        fdy[1]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"Y")  
        fdy[2]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"Y")  
        fdy[3]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"Y")  
        fdy[4]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"Y")  
        fdy[5]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"Y")  
        fdy[6]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"Y")  
        fdy[7]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"Y")  

        fdz[0]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"Z")  
        fdz[1]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"Z")  
        fdz[2]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"Z")  
        fdz[3]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"Z")  
        fdz[4]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"Z")  
        fdz[5]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"Z")  
        fdz[6]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"Z")  
        fdz[7]=FirstOrderDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"Z")  


     
        # Get 2nd Order Mixed Derivatives Nearest Neighbors Values
        fdxdy[0]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"XY")  
        fdxdy[1]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"XY")  
        fdxdy[2]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"XY")  
        fdxdy[3]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"XY")  
        fdxdy[4]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"XY")  
        fdxdy[5]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"XY")  
        fdxdy[6]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"XY")  
        fdxdy[7]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"XY")  

        
        fdxdz[0]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"XZ")  
        fdxdz[1]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"XZ")  
        fdxdz[2]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"XZ")  
        fdxdz[3]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"XZ")  
        fdxdz[4]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"XZ")  
        fdxdz[5]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"XZ")  
        fdxdz[6]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"XZ")  
        fdxdz[7]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"XZ")  
        
        fdydz[0]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"YZ")  
        fdydz[1]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"YZ")  
        fdydz[2]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"YZ")  
        fdydz[3]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"YZ")  
        fdydz[4]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"YZ")  
        fdydz[5]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"YZ")  
        fdydz[6]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"YZ")  
        fdydz[7]=SecondOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"YZ")  

        
        # Get 3rd Order Mixed Derivatives Nearest Neighbors Values

        fdxdydz[0]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local,"XY")  
        fdxdydz[1]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local,"XY")  
        fdxdydz[2]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local,"XY")  
        fdxdydz[3]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local,"XY")  
        fdxdydz[4]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local,phi_local+1,"XY")  
        fdxdydz[5]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local,phi_local+1,"XY")  
        fdxdydz[6]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local,theta_local+1,phi_local+1,"XY")  
        fdxdydz[7]=ThirdOrderMixedDerivative(ne,glat,glon,alts,counter,r_local+1,theta_local+1,phi_local+1,"XY")  


        for ii in range(8):
            x[0+ii]=fval[ii]
            x[8+ii]=fdx[ii]
            x[16+ii]=fdy[ii]
            x[24+ii]=fdz[ii]
            x[32+ii]=fdxdy[ii]
            x[40+ii]=fdxdz[ii]
            x[48+ii]=fdydz[ii]
            x[56+ii]=fdxdydz[ii]


        a =np.zeros(64)
        for ii in range(64):
            a[ii]=0.0
            for j in range(64):
                a[ii]+=A[ii,j]*x[j]

        m[i]=0.0
        m2[i] = ne[handleBC(counter, r_local, theta_local, phi_local)]
        for ii in range(4):
            for j in range(4):
                for k in range(4):
                    m[i]+=a[ii+4*j+16*k]*pow(dx,ii)*pow(dy,j)*pow(dz,k)
   
    m = Tricubic.AdaptiveLPF(m, srt)
    return (m)


def Write(outfile,m,varname):

    ncout = Dataset(outfile, "a", format="NETCDF4")
    data = ncout.createVariable(varname,"f4",('time'))
    data[:]=m
    ncout.close()
    
    
    
    
from ipywidgets import*
import ipywidgets as widgets
import warnings
import sys
sys.path.insert(1, "../SourceCode")

import Tricubic as Tricubic
import DaedalusGlobals as DaedalusGlobals
import glob
import os
import plotly.graph_objects as go
from mpl_toolkits.basemap import Basemap, shiftgrid, addcyclic
import matplotlib.mlab as mlab
from mpl_toolkits.axes_grid1 import make_axes_locatable


def LoadGUI():
   
    layout1 = {'width':'850px'}
    style1 = {'description_width':'150px'}
    tiegcm_file=widgets.Dropdown(options=sorted(glob.glob(DaedalusGlobals.TIEGCM_Files_Path_2 +"TIEGCM_2015/*.nc")), description='TIE-GCM files: ', style=style1, layout=layout1)
    orbit_file=widgets.Dropdown(options=sorted(glob.glob(DaedalusGlobals.Orbit_Files_Path +"DAED_ORB_Evt0_LLA*.nc")), description='Orbit files: ', style=style1, layout=layout1)
    global linearInter,TricubicInter,saveOption,timer_value
    timer_value = widgets.Dropdown(value=22, options=range(0, 24), description='TimeStep:', style=style1,
    layout=layout1)
    saveOption= widgets.Checkbox(value=False,  description="Save Output",style=style1, layout=layout1)
    linearInter= widgets.Checkbox(value=True,  description="Linear Interpolation",style=style1, layout=layout1)
    TricubicInter= widgets.Checkbox(value=True,  description="Tricubic Interpolation",style=style1, layout=layout1)
    TGvar=widgets.Dropdown(
            options=['All','XNMBAR','TI','TN','OP','DEN','SIGMA_HALL','HE','SIGMA_PED','POTEN','TE',
                    'O2_CM3', 'EEX', 'EEY', 'EEZ', 'ELECDEN', 'N2_CM3', 'NOP_LAM', 'NPLUS', '02P', 'O_CM3'
                    'UN', 'VN', 'WN_lev', 'HALL', 'N2D', 'N4S', 'NO', 'Ui_lev', 'Vi_lev', 'WI_LEV','ZGMID',"Ohmic"],
            value='ZGMID',
            description='TIEGCM Variable:',
            disabled=False,
            style=style1, 
            layout=layout1
            )
    def Exec_Btn_Clicked( b ):
        
        print( "Interpolation started..." )
        
        if TGvar.value=='All':
            thisLabel="TN"
            tgvars=['All','XNMBAR','TI','TN','OP','DEN','SIGMA_HALL','HE','SIGMA_PED','POTEN','TE',
                    'O2_CM3', 'EEX', 'EEY', 'EEZ', 'ELECDEN', 'N2_CM3', 'NOP_LAM', 'NPLUS', '02P', 'O_CM3'
                    'UN', 'VN', 'WN_lev', 'HALL', 'N2D', 'N4S', 'NO', 'Ui_lev', 'Vi_lev', 'WI_LEV','ZGMID',"Ohmic"]
            RunInterpolator(os.path.splitext(tiegcm_file.value)[0],os.path.splitext(orbit_file.value)[0],tgvars,saveOption.value,thisLabel)
        else:
            thisLabel="TN"
            tgvars=[TGvar.value]
            if (TGvar.value=="XNMBAR"):
                thisLabel="Conversion Factor "
            if (TGvar.value=="TI"):
                thisLabel="Ion Temperature [K] "
            if (TGvar.value=="TN"):
                thisLabel="Neutral Temperature [K] "
            if (TGvar.value=="OP"):
                thisLabel="O+ ion [cm^-3] "
            if (TGvar.value=="DEN"):
                thisLabel="Total Density [g/cm3] "
            if (TGvar.value=="SIGMA_HALL"):
                thisLabel="Hall conductivity [S/m] "                
            if (TGvar.value=="HE"):
                thisLabel="Helium [mmr] " 
            if (TGvar.value=="SIGMA_PED"):
                thisLabel="Pendersen condcutivity [S/m] "                 
            if (TGvar.value=="POTEN"):
                thisLabel="Electric Potential [Volts] "   
            if (TGvar.value=="TE"):
                thisLabel="Electorn Temperature [K] "     
            if (TGvar.value=="O2_CM3"):
                thisLabel="Molecular Oxygen [cm^-3] "    
            if (TGvar.value=="O2_CM3"):
                thisLabel="Molecular Oxygen [cm^-3] "                           
            if (TGvar.value=="EEX"):
                thisLabel="Electric Field East [V/cm] "
            if (TGvar.value=="EEY"):
                thisLabel="Electric Field North [V/cm] "
            if (TGvar.value=="EEZ"):
                thisLabel="Electric Field Up [V/cm] "
            if (TGvar.value=="ELECDEN"):
                thisLabel="Electron Density [cm^-3] "
            if (TGvar.value=="N2_CM3"):
                thisLabel="Molecular Nitrogen [cm^-3] "
            if (TGvar.value=="NOP_LAM"):
                thisLabel="NO+ Ion [cm^-3] "
            if (TGvar.value=="NPLUS"):
                thisLabel="N+ Ion [cm^-3] "
            if (TGvar.value=="02P"):
                thisLabel="O2+ Ion  [cm^-3] "
            if (TGvar.value=="O_CM3"):
                thisLabel="Atomic Oxygen  [cm^-3] "
            if (TGvar.value=="UN"):
                thisLabel="Neutral Zonal Wind  [cm/s] "
            if (TGvar.value=="VN"):
                thisLabel="Meridional Zonal Wind [cm/s] "
            if (TGvar.value=="WN_lev"):
                thisLabel="Vertical Zonal Wind [cm/s] "
            if (TGvar.value=="HALL"):
                thisLabel="Hall conductivity [S/m] "
            if (TGvar.value=="N2D"):
                thisLabel="N2D [mmr] "
            if (TGvar.value=="N4S"):
                thisLabel="N4S [mmr] "
            if (TGvar.value=="NO"):
                thisLabel="Nitric Oxide [mmr] "
            if (TGvar.value=="Ui_lev"):
                thisLabel="Zonal ExB velocity [cm/s] "
            if (TGvar.value=="Vi_lev"):
                thisLabel="Meridional ExB velocity [cm/s] "
            if (TGvar.value=="WI_LEV"):
                thisLabel="Vertical ExB velocity [cm/s] "
            if (TGvar.value=="ZGMID"):
                thisLabel="ZGMID"
            if (TGvar.value=="Ohmic"):
                thisLabel="Ohmic"

                
            RunInterpolator(os.path.splitext(tiegcm_file.value)[0],os.path.splitext(orbit_file.value)[0],tgvars,saveOption.value,thisLabel)
  

    def createGUI():
        MainPanel = widgets.VBox() 
        MainTab = widgets.Tab() 
        VerticalPanel = widgets.VBox()
        InterpolationPanel=widgets.HBox()
        MapPanel = widgets.VBox()
        MainTab.children = [ VerticalPanel ]
        MainTab.set_title(0, 'Interpolator')
        MainPanel.children = [MainTab ]
        InterpolationPanel.children=[linearInter,TricubicInter]
        Exec_Btn = widgets.Button (description='Run',tooltip="Click here to Interpolate",)
        Exec_Btn.style.button_color = 'Red'
        Exec_Btn.on_click( Exec_Btn_Clicked )
        VerticalPanel.children = [tiegcm_file,orbit_file,timer_value,TGvar,InterpolationPanel,saveOption,Exec_Btn]
    
        return MainPanel
    display( createGUI() )


def RunInterpolator(model_data_file,orbit_file,TGvar,Save=False,thisLabel="TN"):
    model=Model(model_data_file ,500,100)  #initialize model
#     print ("model file ok")
    orbit=Orbit(orbit_file)                #initialize  orbit
#     print("orbit file ok")
    
    outfile=os.getenv("HOME")+"/"+os.path.basename(model_data_file)+"_Interpolated.nc"
    dtime,dlat,dlon,dalt,index,int_final=orbit.createorbit(orbit.name,model.minAltitude,model.maxAltitude,outfile,Save)
#     print("orbit created")
    gtime,glat,glon,glev,zg=model.readGrid(model.name)                                     #get model stats
    

    
    #select variable
    if linearInter.value==True:
        for jj in tqdm(range(len(TGvar))):
            var=model.readVar(model.name,TGvar[jj])
            interpolatedDataTrilinear=Interpolate_Serial(timer_value.value,glat,glon,glev,dtime, dlat,dlon,dalt,zg,var,True,thisLabel)
            interpolatedData=orbit.mergeData(index,int_final,interpolatedDataTrilinear)

            if saveOption.value == True: 
                Write(outfile,interpolatedData,TGvar[jj])
                
    if TricubicInter.value==True:
#         print("tricubic")
        for jj in tqdm(range(len(TGvar))):
            var=model.readVar(model.name,TGvar[jj])
            indexHz=orbit_file.find("Hz")
            srt=float(orbit_file[indexHz-2:indexHz])
            interpolatedDataTricubic=TriCubicSpline(timer_value.value,glat,glon,glev,dtime, dlat,dlon,dalt,zg,var,True,thisLabel,srt)
            interpolatedData=orbit.mergeData(index,int_final,interpolatedDataTricubic)
            
            if saveOption.value == True: 
                Write(outfile,interpolatedData,TGvar[jj])
    if TricubicInter.value==True:
        fig1= go.Figure()
        min=np.zeros((len(dtime)))
#         print(geitonas)
        for i in range(0,len(dtime)):
            min[i] = (dtime[i] - startTime)/1000/60
        fig1.add_trace(go.Scatter(x=min, y=interpolatedDataTricubic,
                name="interpolatedDataTricubic", mode='lines', line=dict(color='brown'))) #dash="dot", visible="legendonly"
#         fig1.add_trace(go.Scatter(x=min, y=geitonas,
#                 name="geitonas", mode='lines', line=dict(color='blue')))

        # updating the layout of the figure. All the parameters concerning graph are put here
        fig1.update_layout( xaxis_showexponent='all', xaxis_exponentformat='power',yaxis_exponentformat='power',
                           yaxis=dict(tickmode='array'), xaxis_title="Time (min)",
                           yaxis_title=thisLabel, width=950, height=750,
                           title={'text': 'Tricubic Interpolated Data (from ' + str(datetime.utcfromtimestamp(startTime/1000.0).strftime('%Y-%m-%d %H:%M:%S')) + ')', 'y': 0.9, 'x': 0.5, #+"<br> Tn "+str(Tn_error.value)+"% error"
                                  'xanchor': 'center', 'yanchor': 'top'})
        fig1.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
        fig1.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
        fig1.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
        fig1.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
        fig1.show()
    
    if linearInter.value==True:
        fig2= go.Figure()
        min=np.zeros((len(dtime)))
        for i in range(0,len(dtime)):
            min[i] = (dtime[i] - startTime)/1000/60
        fig2.add_trace(go.Scatter(x=min, y=interpolatedDataTrilinear,
                name="interpolatedDataTrilinear", mode='lines', line=dict(color='red'))) #dash="dot", visible="legendonly"

        # updating the layout of the figure. All the parameters concerning graph are put here
        fig2.update_layout( xaxis_showexponent='all', xaxis_exponentformat='power',yaxis_exponentformat='power',
                           yaxis=dict(tickmode='array'), xaxis_title="Time (min)",
                           yaxis_title=thisLabel, width=950, height=750,
                           title={'text': 'Trilinear Interpolated Data (from ' + str(datetime.utcfromtimestamp(startTime/1000.0).strftime('%Y-%m-%d %H:%M:%S')) + ')', 'y': 0.9, 'x': 0.5, #+"<br> Tn "+str(Tn_error.value)+"% error"
                                  'xanchor': 'center', 'yanchor': 'top'})
        fig2.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
        fig2.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
        fig2.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
        fig2.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
        fig2.show()
    
    fig3= go.Figure()
    min=np.zeros((len(dtime)))
    for i in range(0,len(dtime)):
        min[i] = (dtime[i] - startTime)/1000/60
    fig3.add_trace(go.Scatter(x=min, y=dlat,
            name="DaedalusLatitude", mode='lines', line=dict(color='blue'))) #dash="dot", visible="legendonly"

    # updating the layout of the figure. All the parameters concerning graph are put here
    fig3.update_layout( xaxis_showexponent='all', xaxis_exponentformat='power',yaxis_exponentformat='power',
                       yaxis=dict(tickmode='array'), xaxis_title="Time (min)",
                       yaxis_title="Lat[deg]", width=950, height=750,
                       title={'text': 'Daedalus Latitude ', 'y': 0.9, 'x': 0.5, #+"<br> Tn "+str(Tn_error.value)+"% error"
                              'xanchor': 'center', 'yanchor': 'top'})
    fig3.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig3.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig3.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
#     fig3.show()
    
    fig4= go.Figure()
    for i in range(0,len(dtime)):
        min[i] = (dtime[i] - startTime)/1000/60
    fig4.add_trace(go.Scatter(x=min, y=dlon,
            name="DaedalusLongitude", mode='lines', line=dict(color='blue'))) #dash="dot", visible="legendonly"

    # updating the layout of the figure. All the parameters concerning graph are put here
    fig4.update_layout( xaxis_showexponent='all', xaxis_exponentformat='power',yaxis_exponentformat='power',
                       yaxis=dict(tickmode='array'), xaxis_title="Time (min)",
                       yaxis_title="Lat[deg]", width=950, height=750,
                       title={'text': 'Daedalus Longtitude ', 'y': 0.9, 'x': 0.5, #+"<br> Tn "+str(Tn_error.value)+"% error"
                              'xanchor': 'center', 'yanchor': 'top'})
    fig4.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig4.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig4.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig4.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig4.show()
    
    fig5= go.Figure()
    min=np.zeros((len(dtime)))
    for i in range(0,len(dtime)):
        min[i] = (dtime[i] - startTime)/1000/60
    fig5.add_trace(go.Scatter(x=min, y=dalt,
            name="DaedalusAltitude", mode='lines', line=dict(color='green'))) #dash="dot", visible="legendonly"

    # updating the layout of the figure. All the parameters concerning graph are put here
    fig5.update_layout( xaxis_showexponent='all', xaxis_exponentformat='power',yaxis_exponentformat='power',
                       yaxis=dict(tickmode='array'), xaxis_title="Time (min)",
                       yaxis_title="Alt[km]", width=950, height=750,
                       title={'text': 'Daedalus Altitude ', 'y': 0.9, 'x': 0.5, #+"<br> Tn "+str(Tn_error.value)+"% error"
                              'xanchor': 'center', 'yanchor': 'top'})
    fig5.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig5.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig5.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig5.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig5.show()
    
    

            

        
  



## A sample call to the Interpolation Module:

In [ ]:
LoadGUI()

